In [38]:
import os
import numpy as np 
import pandas as pd 
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupShuffleSplit, GroupKFold, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score

import xgbfir
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import dtreeviz
from typing import Any, Dict, Union

from yellowbrick import model_selection as ms
from yellowbrick.model_selection import validation_curve

from sklearn import metrics

## All Connectivity Features 

In [58]:
os.chdir('/home/melissa/RESULTS/FINAL_MODEL/Rat/MNEConnectivity/')
coh_df = pd.read_csv('coh_connectivity_dataframe.csv')
pli_df = pd.read_csv('pli_connectivity_dataframe.csv')
plv_df = pd.read_csv('plv_connectivity_dataframe.csv')
wpli_df = pd.read_csv('wpli_connectivity_dataframe.csv')

In [40]:
SYNGAP_1_ID_ls = [ 'S7088', 'S7092', 'S7094', 'S7098', 'S7068', 'S7074', 'S7076', 'S7071', 'S7075', 'S7101']
syngap_2_ls =  ['S7091', 'S7070', 'S7072', 'S7083', 'S7063','S7064', 'S7069', 'S7086'] 
analysis_ls = [ 'S7088', 'S7092', 'S7094', 'S7098', 'S7068', 'S7074', 'S7076', 'S7071', 'S7075', 'S7101',
               'S7091', 'S7070', 'S7072', 'S7083', 'S7063','S7064', 'S7069', 'S7086']

In [41]:
coh_df.drop(['Unnamed: 0'], axis = 1, inplace = True)
pli_df.drop(['Unnamed: 0', 'Animal_ID', 'Idx'], axis = 1, inplace = True )
plv_df.drop(['Unnamed: 0', 'Animal_ID', 'Idx'], axis = 1, inplace = True )
wpli_df.drop(['Unnamed: 0', 'Animal_ID', 'Idx'], axis = 1, inplace = True )

In [42]:
all_dataframes = pd.concat([coh_df, pli_df, plv_df, wpli_df, coh_df], axis = 1)
all_dataframes

,Animal_ID,Idx,Motor_coh_delta,Visual_coh_delta,Somatosensory_coh_delta,Soma_Motor_coh_delta,Vis_Soma_coh_delta,Vis_Mot_coh_delta,Motor_coh_theta,Visual_coh_theta,...,Somatosensory_coh_beta,Soma_Motor_coh_beta,Vis_Soma_coh_beta,Vis_Mot_coh_beta,Motor_coh_gamma,Visual_coh_gamma,Somatosensory_coh_gamma,Soma_Motor_coh_gamma,Vis_Soma_coh_gamma,Vis_Mot_coh_gamma
0,S7088,0,0.446609,0.455715,0.163678,0.116058,0.106383,0.186727,0.437935,0.444330,...,0.159314,0.112658,0.095428,0.176131,0.405608,0.401548,0.155815,0.108974,0.087585,0.165873
1,S7088,1,0.449176,0.455015,0.164865,0.117971,0.108519,0.188251,0.441181,0.453277,...,0.159250,0.108877,0.086915,0.168199,0.398819,0.376802,0.155714,0.103802,0.077507,0.156954
2,S7088,3,0.409097,0.383441,0.159936,0.108380,0.093402,0.159813,0.400871,0.396115,...,0.158709,0.110864,0.091874,0.171432,0.399164,0.382319,0.155241,0.104324,0.082217,0.156760
3,S7088,4,0.417899,0.403514,0.160885,0.110316,0.099708,0.166003,0.413723,0.402761,...,0.158037,0.108555,0.085643,0.163444,0.399362,0.375644,0.154456,0.102679,0.078163,0.152522
4,S7088,5,0.440614,0.458189,0.162612,0.118323,0.109828,0.190117,0.431519,0.441354,...,0.158513,0.109195,0.085029,0.165266,0.402090,0.373355,0.155411,0.103449,0.079004,0.153305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247242,S7086,34552,0.354760,0.311992,0.149577,0.099897,0.062642,0.135210,0.386469,0.401287,...,0.155944,0.101261,0.079330,0.151612,0.388412,0.366301,0.153946,0.098939,0.066684,0.143787
247243,S7086,34555,0.365803,0.380822,0.156422,0.102101,0.088216,0.160846,0.407399,0.418002,...,0.156836,0.104996,0.085226,0.160426,0.388579,0.375112,0.152834,0.099503,0.069467,0.147103
247244,S7086,34556,0.305073,0.324930,0.132031,0.090440,0.072408,0.124794,0.404576,0.405815,...,0.157109,0.097493,0.076560,0.143917,0.395255,0.376303,0.155386,0.099713,0.070883,0.146977
247245,S7086,34558,0.369855,0.370350,0.143726,0.088358,0.075268,0.126779,0.393269,0.406889,...,0.156645,0.100894,0.075253,0.149715,0.385915,0.369435,0.155335,0.098208,0.066915,0.142876


In [ ]:
#all_dataframes.reset_index(inplace = True)

In [ ]:
#all_dataframes = all_dataframes[~duplicates]
#all_dataframes = all_dataframes.reset_index(drop=True)

In [43]:
# Remove duplicate columns
all_dataframes = all_dataframes.loc[:,~all_dataframes.columns.duplicated()]


In [44]:
mot_disp_dir = '/home/melissa/RESULTS/XGBoost/SYNGAP1/Motor/DispEn/'
som_disp_dir = '/home/melissa/RESULTS/XGBoost/SYNGAP1/Somatosensory/DispEn/'
vis_disp_dir = '/home/melissa/RESULTS/XGBoost/SYNGAP1/Visual/DispEn/'
mot_hfd_dir = '/home/melissa/RESULTS/XGBoost/SYNGAP1/Motor/HFD/'
som_hfd_dir = '/home/melissa/RESULTS/XGBoost/SYNGAP1/Somatosensory/HFD/'
vis_hfd_dir = '/home/melissa/RESULTS/XGBoost/SYNGAP1/Visual/HFD/'
mot_hurst_dir = '/home/melissa/RESULTS/XGBoost/SYNGAP1/Motor/Hurst/'
som_hurst_dir = '/home/melissa/RESULTS/XGBoost/SYNGAP1/Somatosensory/Hurst/'
vis_hurst_dir = '/home/melissa/RESULTS/XGBoost/SYNGAP1/Visual/Hurst/'

delta_motor_power_dir = '/home/melissa/RESULTS/XGBoost/SYNGAP1/Motor/Delta_Power/' 
theta_motor_power_dir = '/home/melissa/RESULTS/XGBoost/SYNGAP1/Motor/Theta_Power/' 
sigma_motor_power_dir = '/home/melissa/RESULTS/XGBoost/SYNGAP1/Motor/Sigma_Power/' 
beta_motor_power_dir = '/home/melissa/RESULTS/XGBoost/SYNGAP1/Motor/Beta_Power/' 
gamma_motor_power_dir = '/home/melissa/RESULTS/XGBoost/SYNGAP1/Motor/Gamma_Power/' 

delta_soma_power_dir = '/home/melissa/RESULTS/XGBoost/SYNGAP1/Somatosensory/Delta_Power/' 
theta_soma_power_dir = '/home/melissa/RESULTS/XGBoost/SYNGAP1/Somatosensory/Theta_Power/' 
sigma_soma_power_dir = '/home/melissa/RESULTS/XGBoost/SYNGAP1/Somatosensory/Sigma_Power/' 
beta_soma_power_dir = '/home/melissa/RESULTS/XGBoost/SYNGAP1/Somatosensory/Beta_Power/' 
gamma_soma_power_dir = '/home/melissa/RESULTS/XGBoost/SYNGAP1/Somatosensory/Gamma_Power/'

delta_visual_power_dir = '/home/melissa/RESULTS/XGBoost/SYNGAP1/Visual/Delta_Power/' 
theta_visual_power_dir = '/home/melissa/RESULTS/XGBoost/SYNGAP1/Visual/Theta_Power/' 
sigma_visual_power_dir = '/home/melissa/RESULTS/XGBoost/SYNGAP1/Visual/Sigma_Power/' 
beta_visual_power_dir = '/home/melissa/RESULTS/XGBoost/SYNGAP1/Visual/Beta_Power/' 
gamma_visual_power_dir = '/home/melissa/RESULTS/XGBoost/SYNGAP1/Visual/Gamma_Power/'


mot_cc_delta_dir = '/home/melissa/RESULTS/XGBoost/SYNGAP1/Motor/CrossCorr_Motor_Delta/'
mot_cc_theta_dir = '/home/melissa/RESULTS/XGBoost/SYNGAP1/Motor/CrossCorr_Motor_Theta/'
mot_cc_sigma_dir = '/home/melissa/RESULTS/XGBoost/SYNGAP1/Motor/CrossCorr_Motor_Sigma/'
mot_cc_beta_dir = '/home/melissa/RESULTS/XGBoost/SYNGAP1/Motor/CrossCorr_Motor_Beta/'
mot_cc_gamma_dir = '/home/melissa/RESULTS/XGBoost/SYNGAP1/Motor/CrossCorr_Motor_Gamma/'

som_cc_delta_dir = '/home/melissa/RESULTS/XGBoost/SYNGAP1/Somatosensory/CrossCorr_Som_Delta/'
som_cc_theta_dir = '/home/melissa/RESULTS/XGBoost/SYNGAP1/Somatosensory/CrossCorr_Som_Theta/'
som_cc_sigma_dir = '/home/melissa/RESULTS/XGBoost/SYNGAP1/Somatosensory/CrossCorr_Som_Sigma/'
som_cc_beta_dir = '/home/melissa/RESULTS/XGBoost/SYNGAP1/Somatosensory/CrossCorr_Som_Beta/'
som_cc_gamma_dir = '/home/melissa/RESULTS/XGBoost/SYNGAP1/Somatosensory/CrossCorr_Som_Gamma/'

vis_cc_delta_dir = '/home/melissa/RESULTS/XGBoost/SYNGAP1/Visual/CrossCorr_Vis_Delta/'
vis_cc_theta_dir = '/home/melissa/RESULTS/XGBoost/SYNGAP1/Visual/CrossCorr_Vis_Theta/'
vis_cc_sigma_dir = '/home/melissa/RESULTS/XGBoost/SYNGAP1/Visual/CrossCorr_Vis_Sigma/'
vis_cc_beta_dir = '/home/melissa/RESULTS/XGBoost/SYNGAP1/Visual/CrossCorr_Vis_Beta/'
vis_cc_gamma_dir = '/home/melissa/RESULTS/XGBoost/SYNGAP1/Visual/CrossCorr_Vis_Gamma/'


clean_br_dir = '/home/melissa/PREPROCESSING/SYNGAP1/br_files_XGBoost/'

In [47]:
analysis_ls = [ 'S7088', 'S7092', 'S7094', 'S7098', 'S7068', 'S7074', 'S7076', 'S7071', 'S7075', 'S7101',
                 'S7083', 'S7063','S7064', 'S7069', 'S7086', 'S7091', 'S7070', 'S7072']

one_br_list = [ 'S7088', 'S7092', 'S7094' , 'S7098', 'S7068', 'S7074', 'S7076', 'S7071', 'S7075', 'S7101']
two_br_list = ['S7070','S7072','S7083', 'S7063','S7064','S7069', 'S7086','S7091', 'S7096']


In [90]:
all_animals_df = []
for animal in analysis_ls:
    print(animal)
    #mne_connect_df = all_dataframes[all_dataframes['Animal_ID'] == str(animal)]
    #print(mne_connect_df)
    
    if animal in two_br_list:
        clean_br_file_1 = pd.read_csv(clean_br_dir + animal + '_BL1.csv')
        clean_br_file_2 = pd.read_csv(clean_br_dir + animal + '_BL2.csv')
        sleepstate_1 = np.array(clean_br_file_1['brainstate'].to_list())
        sleepstate_2 = np.array(clean_br_file_2['brainstate'].to_list())
        all_indices_1 = np.where((sleepstate_1 == 0) | (sleepstate_1 == 1) | (sleepstate_1 == 2))[0]
        all_indices_2_old = np.where((sleepstate_2 == 0) | (sleepstate_2 == 1) | (sleepstate_2 == 2))[0]
        all_indices_2 = [idx + 17280 for idx in all_indices_2_old]
        clean_indices = np.concatenate((all_indices_1, all_indices_2))
        
    elif animal in one_br_list:
        clean_br = pd.read_csv(clean_br_dir + animal + '_BL1.csv')
        sleepstate = np.array(clean_br['brainstate'].to_list())
        clean_indices = np.where((sleepstate == 0) | (sleepstate == 1) | (sleepstate == 2))[0]
    
    else:
        print(animal + ' no clean indices')
    
    mot_disp = np.load(mot_disp_dir + animal + '_dispen.npy')
    som_disp = np.load(som_disp_dir + animal + '_dispen.npy')
    vis_disp = np.load(vis_disp_dir + animal + '_dispen.npy' )
    
    mot_hfd = np.load(mot_hfd_dir + animal + '_hfd_concat.npy')
    som_hfd = np.load(som_hfd_dir + animal + '_hfd_concat.npy')
    vis_hfd = np.load(vis_hfd_dir + animal + '_hfd_concat.npy' )
    
    mot_hfd_concat = np.array([val[0] for val in mot_hfd])
    som_hfd_concat = np.array([val[0] for val in som_hfd])
    vis_hfd_concat = np.array([val[0] for val in vis_hfd])
    
    mot_hurst = np.load(mot_hurst_dir + animal + '_hurst_concat.npy')
    som_hurst = np.load(som_hurst_dir + animal + '_hurst_concat.npy')
    vis_hurst = np.load(vis_hurst_dir + animal + '_hurst_concat.npy' )
    
    mot_hurst_concat = np.array([val[0] for val in mot_hurst])
    som_hurst_concat = np.array([val[0] for val in som_hurst])
    vis_hurst_concat = np.array([val[0] for val in vis_hurst])
    
    mot_cc_delta_left = np.load(mot_cc_delta_dir + animal + '_mot_left_cross_cor.npy')
    mot_cc_delta_right = np.load(mot_cc_delta_dir + animal + '_mot_right_cross_cor.npy')
    mot_cc_delta = np.mean(np.stack((mot_cc_delta_left, mot_cc_delta_right)), axis=0)
    som_cc_delta_left = np.load(som_cc_delta_dir + animal + '_som_left_cross_cor.npy')
    som_cc_delta_right = np.load(som_cc_delta_dir + animal + '_som_right_cross_cor.npy')
    som_cc_delta = np.mean(np.stack((som_cc_delta_left, som_cc_delta_right)), axis=0)
    vis_cc_delta_left = np.load(vis_cc_delta_dir + animal + '_vis_left_cross_cor.npy')
    vis_cc_delta_right = np.load(vis_cc_delta_dir + animal + '_vis_right_cross_cor.npy')
    vis_cc_delta = np.mean(np.stack((vis_cc_delta_left, vis_cc_delta_right)), axis=0)
    
    
    mot_cc_theta_left = np.load(mot_cc_theta_dir + animal + '_mot_left_cross_cor.npy')
    mot_cc_theta_right = np.load(mot_cc_theta_dir + animal + '_mot_right_cross_cor.npy')
    mot_cc_theta = np.mean(np.stack((mot_cc_theta_left, mot_cc_theta_right)), axis=0)
    som_cc_theta_left = np.load(som_cc_theta_dir + animal + '_som_left_cross_cor.npy')
    som_cc_theta_right = np.load(som_cc_theta_dir + animal + '_som_right_cross_cor.npy')
    som_cc_theta = np.mean(np.stack((som_cc_theta_left, som_cc_theta_right)), axis=0)
    vis_cc_theta_left = np.load(vis_cc_theta_dir + animal + '_vis_left_cross_cor.npy')
    vis_cc_theta_right = np.load(vis_cc_theta_dir + animal + '_vis_right_cross_cor.npy')
    vis_cc_theta = np.mean(np.stack((vis_cc_theta_left, vis_cc_theta_right)), axis=0)
    
    
    mot_cc_sigma_left = np.load(mot_cc_sigma_dir + animal + '_mot_left_cross_cor.npy')
    mot_cc_sigma_right = np.load(mot_cc_sigma_dir + animal + '_mot_right_cross_cor.npy')
    mot_cc_sigma = np.mean(np.stack((mot_cc_sigma_left, mot_cc_sigma_right)), axis=0)
    som_cc_sigma_left = np.load(som_cc_sigma_dir + animal + '_som_left_cross_cor.npy')
    som_cc_sigma_right = np.load(som_cc_sigma_dir + animal + '_som_right_cross_cor.npy')
    som_cc_sigma = np.mean(np.stack((som_cc_sigma_left, som_cc_sigma_right)), axis=0)
    vis_cc_sigma_left = np.load(vis_cc_sigma_dir + animal + '_vis_left_cross_cor.npy')
    vis_cc_sigma_right = np.load(vis_cc_sigma_dir + animal + '_vis_right_cross_cor.npy')
    vis_cc_sigma = np.mean(np.stack((vis_cc_sigma_left, vis_cc_sigma_right)), axis=0)
    
    mot_cc_beta_left = np.load(mot_cc_beta_dir + animal + '_mot_left_cross_cor.npy')
    mot_cc_beta_right = np.load(mot_cc_beta_dir + animal + '_mot_right_cross_cor.npy')
    mot_cc_beta = np.mean(np.stack((mot_cc_beta_left, mot_cc_beta_right)), axis=0)
    som_cc_beta_left = np.load(som_cc_beta_dir + animal + '_som_left_cross_cor.npy')
    som_cc_beta_right = np.load(som_cc_beta_dir + animal + '_som_right_cross_cor.npy')
    som_cc_beta = np.mean(np.stack((som_cc_beta_left, som_cc_beta_right)), axis=0)
    vis_cc_beta_left = np.load(vis_cc_beta_dir + animal + '_vis_left_cross_cor.npy')
    vis_cc_beta_right = np.load(vis_cc_beta_dir + animal + '_vis_right_cross_cor.npy')
    vis_cc_beta = np.mean(np.stack((vis_cc_beta_left, vis_cc_beta_right)), axis=0)

    mot_cc_gamma_left = np.load(mot_cc_gamma_dir + animal + '_mot_left_cross_cor.npy')
    mot_cc_gamma_right = np.load(mot_cc_gamma_dir + animal + '_mot_right_cross_cor.npy')
    mot_cc_gamma = np.mean(np.stack((mot_cc_gamma_left, mot_cc_gamma_right)), axis=0)
    som_cc_gamma_right = np.load(som_cc_gamma_dir + animal + '_som_right_cross_cor.npy')
    som_cc_gamma_left = np.load(som_cc_gamma_dir + animal + '_som_left_cross_cor.npy')
    som_cc_gamma = np.mean(np.stack((som_cc_gamma_left, som_cc_gamma_right)), axis=0)
    vis_cc_gamma_left = np.load(vis_cc_gamma_dir + animal + '_vis_left_cross_cor.npy')
    vis_cc_gamma_right = np.load(vis_cc_gamma_dir + animal + '_vis_right_cross_cor.npy')
    vis_cc_gamma = np.mean(np.stack((vis_cc_gamma_left, vis_cc_gamma_right)), axis=0)
    
    mot_delta_power = np.load(delta_motor_power_dir + animal + '_power.npy')
    mot_theta_power = np.load(theta_motor_power_dir + animal + '_power.npy')
    mot_sigma_power = np.load(sigma_motor_power_dir + animal + '_power.npy')
    mot_beta_power = np.load(beta_motor_power_dir + animal + '_power.npy')
    mot_gamma_power = np.load(gamma_motor_power_dir + animal + '_power.npy')
    
    som_delta_power = np.load(delta_soma_power_dir + animal + '_power.npy')
    som_theta_power = np.load(delta_soma_power_dir + animal + '_power.npy')
    som_sigma_power = np.load(delta_soma_power_dir + animal + '_power.npy')
    som_beta_power = np.load(delta_soma_power_dir + animal + '_power.npy')
    som_gamma_power = np.load(delta_soma_power_dir + animal + '_power.npy')
    
    vis_delta_power = np.load(delta_visual_power_dir + animal + '_power.npy')
    vis_theta_power = np.load(theta_visual_power_dir + animal + '_power.npy')
    vis_sigma_power = np.load(sigma_visual_power_dir + animal + '_power.npy')
    vis_beta_power = np.load(beta_visual_power_dir + animal + '_power.npy')
    vis_gamma_power = np.load(gamma_visual_power_dir + animal + '_power.npy')
    

    
    complexity_dict = {'Animal_ID': [str(animal)] *len(vis_cc_gamma),
                       'Idx': np.arange(0, len(vis_cc_gamma), 1),
                       'Mot_Disp': mot_disp, 'Som_Disp': som_disp, 'Vis_Disp': vis_disp,
                    'Mot_HFD': mot_hfd_concat, 'Som_HFD': som_hfd_concat, 'Vis_HFD': vis_hfd_concat,
                    'Mot_Hurst': mot_hurst_concat, 'Som_Hurst': som_hurst_concat, 'Vis_Hurst': vis_hurst_concat,
                    'Mot_CC_Delta': mot_cc_delta, 'Som_CC_Delta': som_cc_delta, 'Vis_CC_Delta': vis_cc_delta,
                    'Mot_CC_Theta': mot_cc_theta, 'Som_CC_Theta': som_cc_theta, 'Vis_CC_Theta': vis_cc_theta,
                    'Mot_CC_Sigma': mot_cc_sigma, 'Som_CC_Sigma': som_cc_sigma, 'Vis_CC_Sigma': vis_cc_sigma, 
                    'Mot_CC_Beta': mot_cc_beta, 'Som_CC_Beta': som_cc_beta, 'Vis_CC_Beta': vis_cc_beta, 
                    'Mot_CC_Gamma': mot_cc_gamma, 'Som_CC_Gamma': som_cc_gamma, 'Vis_CC_Gamma': vis_cc_gamma,
                    'Mot_Delta': mot_delta_power, 'Mot_Theta': mot_theta_power, 'Mot_Sigma': mot_sigma_power,
                    'Mot_Beta': mot_beta_power, 'Mot_Gamma': mot_gamma_power,
                    'Som_Delta': mot_delta_power, 'Som_Theta': mot_theta_power, 'Som_Sigma': mot_sigma_power,
                    'Som_Beta': mot_beta_power, 'Som_Gamma': mot_gamma_power,
                    'Vis_Delta': mot_delta_power, 'Vis_Theta': mot_theta_power, 'Vis_Sigma': mot_sigma_power,
                    'Vis_Beta': mot_beta_power, 'Vis_Gamma': mot_gamma_power}
    
    
    complexity_df = pd.DataFrame(data = complexity_dict)
    print(len(complexity_df))
    filtered_df = complexity_df[complexity_df['Idx'].isin(clean_indices)]
    print(len(filtered_df))
    #indices = mne_connect_df['Idx'].to_list()
    #selected_rows = complexity_df.loc[indices]
    #test_concat = pd.concat([mne_connect_df, selected_rows], axis = 1)
    #complexity_df_cleaned = test_concat.dropna()
    all_animals_df.append(filtered_df)

S7088
17280
168
S7092
17280
857
S7094
17280
1789
S7098
17280
35
S7068
17280
294
S7074
17280
24
S7076
17280
211
S7071
17280
1138
S7075
17280
4
S7101
17280
1971
S7083
34560
330
S7063
34560
1855
S7064
34560
705
S7069
34560
290
S7086
34560
516
S7091
34560
2455
S7070
34560
5332
S7072
34560
5588


In [86]:
concat_animals_df = pd.concat(all_animals_df)
concat_animals_df.to_csv('/home/melissa/RESULTS/FINAL_MODEL/Rat/' + 'rat_xgboost_clean_indices.csv')

In [59]:
concat_animals

,Unnamed: 0,Animal_ID,Idx,Motor_plv_delta,Visual_plv_delta,Somatosensory_plv_delta,Soma_Motor_plv_delta,Vis_Soma_plv_delta,Vis_Mot_plv_delta,Motor_plv_theta,...,Somatosensory_plv_beta,Soma_Motor_plv_beta,Vis_Soma_plv_beta,Vis_Mot_plv_beta,Motor_plv_gamma,Visual_plv_gamma,Somatosensory_plv_gamma,Soma_Motor_plv_gamma,Vis_Soma_plv_gamma,Vis_Mot_plv_gamma
0,0,S7088,0,0.388028,0.414614,0.156792,0.104698,0.086078,0.168925,0.406710,...,0.153077,0.102368,0.080384,0.154229,0.368962,0.365655,0.145756,0.099548,0.076095,0.148402
1,0,S7088,1,0.406497,0.387735,0.148194,0.101800,0.085526,0.162830,0.406686,...,0.150531,0.100503,0.078920,0.153538,0.358214,0.329165,0.143173,0.092060,0.063967,0.137209
2,0,S7088,3,0.396796,0.380947,0.159876,0.106488,0.093902,0.155068,0.373555,...,0.148067,0.102428,0.080624,0.156708,0.360944,0.331127,0.143090,0.092955,0.065162,0.135002
3,0,S7088,4,0.344717,0.349445,0.153412,0.096304,0.091842,0.127331,0.380256,...,0.144539,0.097045,0.069273,0.142156,0.355917,0.319339,0.140840,0.089948,0.061441,0.127648
4,0,S7088,5,0.424419,0.435428,0.155659,0.111965,0.100882,0.177274,0.423002,...,0.150447,0.101198,0.076162,0.150301,0.366180,0.331899,0.143563,0.094052,0.068971,0.137728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247242,0,S7086,34552,0.354626,0.308035,0.145084,0.097234,0.059309,0.129336,0.354890,...,0.146153,0.088839,0.069380,0.132595,0.337225,0.314857,0.141489,0.084116,0.051200,0.119826
247243,0,S7086,34555,0.314267,0.351502,0.152299,0.088724,0.078059,0.144680,0.376989,...,0.147835,0.094219,0.073905,0.140300,0.347646,0.319506,0.141049,0.087351,0.057377,0.124645
247244,0,S7086,34556,0.284220,0.302162,0.111369,0.075788,0.063960,0.110638,0.373800,...,0.147962,0.086082,0.064826,0.123248,0.354977,0.324508,0.145690,0.086588,0.055966,0.123327
247245,0,S7086,34558,0.316314,0.334758,0.133753,0.078374,0.066892,0.109704,0.378360,...,0.147151,0.089201,0.063284,0.130966,0.352984,0.325657,0.146975,0.088488,0.057367,0.127713


## Save original features

In [82]:
all_measures_ls = []
for animal in analysis_ls:
    print(animal)
    connect_indices = concat_animals_df[concat_animals_df['Animal_ID'] == str(animal)]['Idx'].to_list()
    plv_df_animal = plv_df[plv_df['Animal_ID'] == str(animal)]
    animal_complex_df = concat_animals_df.loc[concat_animals_df['Animal_ID'] == str(animal)]
    filtered_df = plv_df_animal[plv_df_animal.index.isin(connect_indices)]
    filtered_df.reset_index(inplace = True)
    animal_complex_df.reset_index(inplace = True)
    all_measures = pd.concat([filtered_df, animal_complex_df], axis = 1)
    all_measures_ls.append(all_measures)

S7088
     index  Unnamed: 0 Animal_ID     Idx  Motor_plv_delta  Visual_plv_delta  \
0      0.0         0.0     S7088     0.0         0.388028          0.414614   
1    176.0         0.0     S7088   335.0         0.409963          0.349800   
2    232.0         0.0     S7088   403.0         0.403958          0.421097   
3    548.0         0.0     S7088   883.0         0.444452          0.450129   
4    708.0         0.0     S7088  1091.0         0.384428          0.419942   
..     ...         ...       ...     ...              ...               ...   
163    NaN         NaN       NaN     NaN              NaN               NaN   
164    NaN         NaN       NaN     NaN              NaN               NaN   
165    NaN         NaN       NaN     NaN              NaN               NaN   
166    NaN         NaN       NaN     NaN              NaN               NaN   
167    NaN         NaN       NaN     NaN              NaN               NaN   

     Somatosensory_plv_delta  Soma_Motor_plv_

   index  Unnamed: 0 Animal_ID  Idx  Motor_plv_delta  Visual_plv_delta  \
0    NaN         NaN       NaN  NaN              NaN               NaN   
1    NaN         NaN       NaN  NaN              NaN               NaN   
2    NaN         NaN       NaN  NaN              NaN               NaN   
3    NaN         NaN       NaN  NaN              NaN               NaN   

   Somatosensory_plv_delta  Soma_Motor_plv_delta  Vis_Soma_plv_delta  \
0                      NaN                   NaN                 NaN   
1                      NaN                   NaN                 NaN   
2                      NaN                   NaN                 NaN   
3                      NaN                   NaN                 NaN   

   Vis_Mot_plv_delta  ...     Som_Delta  Som_Theta  Som_Sigma  Som_Beta  \
0                NaN  ...    245.043207  27.510242   2.893692  1.094444   
1                NaN  ...    571.532081  29.981672  12.437413  2.499528   
2                NaN  ...  46335.839321  18

      index  Unnamed: 0 Animal_ID  Idx  Motor_plv_delta  Visual_plv_delta  \
0       NaN         NaN       NaN  NaN              NaN               NaN   
1       NaN         NaN       NaN  NaN              NaN               NaN   
2       NaN         NaN       NaN  NaN              NaN               NaN   
3       NaN         NaN       NaN  NaN              NaN               NaN   
4       NaN         NaN       NaN  NaN              NaN               NaN   
...     ...         ...       ...  ...              ...               ...   
5327    NaN         NaN       NaN  NaN              NaN               NaN   
5328    NaN         NaN       NaN  NaN              NaN               NaN   
5329    NaN         NaN       NaN  NaN              NaN               NaN   
5330    NaN         NaN       NaN  NaN              NaN               NaN   
5331    NaN         NaN       NaN  NaN              NaN               NaN   

      Somatosensory_plv_delta  Soma_Motor_plv_delta  Vis_Soma_plv_delta  \


In [78]:
all_measures_concat = pd.concat(all_measures_ls)
all_measures_concat.drop(['index'], axis = 1, inplace = True)

In [79]:
clean_all_measures = all_measures_concat.dropna()
clean_all_measures

,Unnamed: 0,Animal_ID,Idx,Motor_plv_delta,Visual_plv_delta,Somatosensory_plv_delta,Soma_Motor_plv_delta,Vis_Soma_plv_delta,Vis_Mot_plv_delta,Motor_plv_theta,...,Som_Delta,Som_Theta,Som_Sigma,Som_Beta,Som_Gamma,Vis_Delta,Vis_Theta,Vis_Sigma,Vis_Beta,Vis_Gamma
0,0.0,S7088,0.0,0.388028,0.414614,0.156792,0.104698,0.086078,0.168925,0.406710,...,134.793524,54.641956,21.328116,3.948171,1.460645,134.793524,54.641956,21.328116,3.948171,1.460645
1,0.0,S7088,335.0,0.409963,0.349800,0.157233,0.107273,0.084750,0.154122,0.387994,...,132.874300,7.122581,2.229062,1.448138,0.988153,132.874300,7.122581,2.229062,1.448138,0.988153
2,0.0,S7088,403.0,0.403958,0.421097,0.155153,0.107986,0.096864,0.174534,0.377130,...,46.986721,9.520694,3.255994,2.125094,1.543682,46.986721,9.520694,3.255994,2.125094,1.543682
3,0.0,S7088,883.0,0.444452,0.450129,0.161451,0.116546,0.107707,0.184904,0.388977,...,678.251029,46.610585,15.499732,2.296411,0.628882,678.251029,46.610585,15.499732,2.296411,0.628882
4,0.0,S7088,1091.0,0.384428,0.419942,0.155375,0.110191,0.092487,0.164921,0.359532,...,447.087895,106.243955,24.430055,5.809671,0.569876,447.087895,106.243955,24.430055,5.809671,0.569876
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,0.0,S7094,3512.0,0.402177,0.419001,0.158195,0.115065,0.090542,0.179616,0.401533,...,1942.972026,33.689895,7.030597,4.177077,1.199805,1942.972026,33.689895,7.030597,4.177077,1.199805
350,0.0,S7094,3527.0,0.409967,0.402966,0.153894,0.111056,0.089734,0.167370,0.388659,...,2322.892381,19.372442,5.403345,3.657544,1.121151,2322.892381,19.372442,5.403345,3.657544,1.121151
351,0.0,S7094,3530.0,0.406895,0.400196,0.152255,0.107418,0.085479,0.171212,0.384831,...,85.780263,15.357691,7.915897,5.202599,2.843704,85.780263,15.357691,7.915897,5.202599,2.843704
352,0.0,S7094,3533.0,0.347529,0.375465,0.137899,0.097567,0.070899,0.147090,0.389006,...,13719.717575,9767.665992,7809.396312,3826.942869,311.819783,13719.717575,9767.665992,7809.396312,3826.942869,311.819783


In [27]:
#os.chdir('/home/melissa/RESULTS/FINAL_MODEL/Rat/')
#all_measures_concat.to_csv('original_rat_features.csv')

In [21]:
#test_concat.to_csv('/home/melissa/RESULTS/FINAL_MODEL/Rat/Complexity/' + 'all_animals_complexity_cc.csv')

In [36]:
os.chdir('/home/melissa/RESULTS/FINAL_MODEL/Rat/MNEConnectivity/')
coh_df = pd.read_csv('coh_connectivity_dataframe.csv')
pli_df = pd.read_csv('pli_connectivity_dataframe.csv')
plv_df = pd.read_csv('plv_connectivity_dataframe.csv')
wpli_df = pd.read_csv('wpli_connectivity_dataframe.csv')

In [66]:
all_measures_ls = []
for animal in analysis_ls:
    clean_indices = concat_animals_df[concat_animals_df['Animal_ID'] == str(animal)]['Idx'].to_list()
    mne_connect_df = all_dataframes[all_dataframes['Animal_ID'] == str(animal)]
    animal_complex_df = concat_animals_df.loc[concat_animals_df['Animal_ID'] == str(animal)]
    filtered_df = mne_connect_df[mne_connect_df.index.isin(connect_indices)]
    filtered_df.reset_index(inplace = True)
    animal_complex_df.reset_index(inplace = True)
    all_measures = pd.concat([filtered_df, animal_complex_df], axis = 1)
    print(all_measures)
    all_measures_ls.append(all_measures)
    

      index Animal_ID    Idx  Motor_coh_delta  Visual_coh_delta  \
0        53     S7088    140         0.442872          0.450676   
1        54     S7088    141         0.433773          0.422447   
2        62     S7088    150         0.408345          0.376004   
3        64     S7088    152         0.422139          0.443847   
4        67     S7088    156         0.434190          0.407037   
...     ...       ...    ...              ...               ...   
1631   7265     S7088  17041         0.429662          0.416449   
1632   7271     S7088  17047         0.439412          0.446911   
1633   7276     S7088  17053         0.436062          0.436364   
1634   7281     S7088  17059         0.433867          0.432723   
1635   7283     S7088  17061         0.440700          0.453359   

      Somatosensory_coh_delta  Soma_Motor_coh_delta  Vis_Soma_coh_delta  \
0                    0.162434              0.116142            0.103729   
1                    0.162856              0.

   index Animal_ID  Idx  Motor_coh_delta  Visual_coh_delta  \
0    NaN       NaN  NaN              NaN               NaN   
1    NaN       NaN  NaN              NaN               NaN   
2    NaN       NaN  NaN              NaN               NaN   
3    NaN       NaN  NaN              NaN               NaN   

   Somatosensory_coh_delta  Soma_Motor_coh_delta  Vis_Soma_coh_delta  \
0                      NaN                   NaN                 NaN   
1                      NaN                   NaN                 NaN   
2                      NaN                   NaN                 NaN   
3                      NaN                   NaN                 NaN   

   Vis_Mot_coh_delta  Motor_coh_theta  ...     Som_Delta  Som_Theta  \
0                NaN              NaN  ...    245.043207  27.510242   
1                NaN              NaN  ...    571.532081  29.981672   
2                NaN              NaN  ...  46335.839321  18.061905   
3                NaN              NaN  ...  

      index Animal_ID  Idx  Motor_coh_delta  Visual_coh_delta  \
0       NaN       NaN  NaN              NaN               NaN   
1       NaN       NaN  NaN              NaN               NaN   
2       NaN       NaN  NaN              NaN               NaN   
3       NaN       NaN  NaN              NaN               NaN   
4       NaN       NaN  NaN              NaN               NaN   
...     ...       ...  ...              ...               ...   
5327    NaN       NaN  NaN              NaN               NaN   
5328    NaN       NaN  NaN              NaN               NaN   
5329    NaN       NaN  NaN              NaN               NaN   
5330    NaN       NaN  NaN              NaN               NaN   
5331    NaN       NaN  NaN              NaN               NaN   

      Somatosensory_coh_delta  Soma_Motor_coh_delta  Vis_Soma_coh_delta  \
0                         NaN                   NaN                 NaN   
1                         NaN                   NaN                 N

In [67]:
all_measures_concat = pd.concat(all_measures_ls)

In [74]:
all_measures_concat = all_measures_concat.loc[:,~all_measures_concat.columns.duplicated()]

In [75]:
all_measures_concat.drop(['index'], axis = 1, inplace = True)
all_measures_concat

KeyError: "['index'] not found in axis"

In [72]:
all_measures_clean = all_measures_concat.dropna()
all_measures_clean

,Animal_ID,Idx,Motor_coh_delta,Visual_coh_delta,Somatosensory_coh_delta,Soma_Motor_coh_delta,Vis_Soma_coh_delta,Vis_Mot_coh_delta,Motor_coh_theta,Visual_coh_theta,...,Som_Delta,Som_Theta,Som_Sigma,Som_Beta,Som_Gamma,Vis_Delta,Vis_Theta,Vis_Sigma,Vis_Beta,Vis_Gamma
0,S7088,140.0,0.442872,0.450676,0.162434,0.116142,0.103729,0.184231,0.434545,0.431143,...,134.793524,54.641956,21.328116,3.948171,1.460645,134.793524,54.641956,21.328116,3.948171,1.460645
1,S7088,141.0,0.433773,0.422447,0.162856,0.115385,0.097571,0.178656,0.433304,0.434425,...,132.874300,7.122581,2.229062,1.448138,0.988153,132.874300,7.122581,2.229062,1.448138,0.988153
2,S7088,150.0,0.408345,0.376004,0.158844,0.106243,0.071448,0.150107,0.412203,0.401653,...,46.986721,9.520694,3.255994,2.125094,1.543682,46.986721,9.520694,3.255994,2.125094,1.543682
3,S7088,152.0,0.422139,0.443847,0.160166,0.114422,0.100890,0.179766,0.432591,0.432616,...,678.251029,46.610585,15.499732,2.296411,0.628882,678.251029,46.610585,15.499732,2.296411,0.628882
4,S7088,156.0,0.434190,0.407037,0.160437,0.111843,0.092280,0.165043,0.425680,0.416720,...,447.087895,106.243955,24.430055,5.809671,0.569876,447.087895,106.243955,24.430055,5.809671,0.569876
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,S7068,3144.0,0.436368,0.429766,0.161537,0.112738,0.096263,0.182285,0.432142,0.417742,...,263.758498,41.709774,12.117661,4.345205,1.743121,263.758498,41.709774,12.117661,4.345205,1.743121
290,S7068,3145.0,0.437717,0.437317,0.162762,0.115666,0.102223,0.182005,0.434642,0.434238,...,213.361116,95.483597,55.701308,19.310435,7.039251,213.361116,95.483597,55.701308,19.310435,7.039251
291,S7068,3149.0,0.453981,0.468983,0.164965,0.119096,0.113862,0.194443,0.443140,0.442795,...,270.416099,33.686842,16.196967,3.492132,1.482258,270.416099,33.686842,16.196967,3.492132,1.482258
292,S7068,3155.0,0.428955,0.419405,0.161388,0.112425,0.087558,0.177546,0.436327,0.434680,...,261.443918,22.814487,8.962767,2.906398,1.204430,261.443918,22.814487,8.962767,2.906398,1.204430


## Save new dataframe with complexity and all mne features

In [76]:
all_measures_clean.to_csv('/home/melissa/RESULTS/FINAL_MODEL/Rat/' + 'all_measures_xgboost_clean_indices.csv')